In [1]:
# 개발환경(OS) : colab(Linux)

In [ ]:
# python==3.7.12
# albumentations==1.1.0
# numpy==1.19.5
# pandas==1.3.5
# cv2==4.1.2
# sklearn==1.0.2
# json==2.0.9
# torch==1.10.0+cu111
# timm==0.5.4
# transformers==4.16.2

In [2]:
# import google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# pakage

!pip uninstall opencv-python-headless==4.5.5.62 --yes
!pip install opencv-python-headless==4.1.2.30

import pickle
import gc
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
from sklearn.utils import shuffle
import random
import os
import json 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import models
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

!pip install timm
!pip install -U albumentations 

import timm
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

os.environ["CUDA_VISIBLE_DEVICES"]="0"

     |████████████████████████████████| 21.8 MB 103.9 MB/s 
     |████████████████████████████████| 431 kB 5.0 MB/s 
     |████████████████████████████████| 102 kB 6.6 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [4]:
# unzip train data

!unzip "/content/drive/MyDrive/Dacon/농업 환경 변화에 따른 작물 병해 진단 AI 경진대회/data/train.zip"
!unzip "/content/drive/MyDrive/Dacon/농업 환경 변화에 따른 작물 병해 진단 AI 경진대회/data/test.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: test/65823/65823.csv    
  inflating: test/65823/65823.jpg    
   creating: test/65824/
  inflating: test/65824/65824.jpg    
  inflating: test/65824/65824.csv    
   creating: test/65825/
  inflating: test/65825/65825.csv    
  inflating: test/65825/65825.jpg    
   creating: test/65827/
  inflating: test/65827/65827.jpg    
  inflating: test/65827/65827.csv    
   creating: test/65828/
  inflating: test/65828/65828.jpg    
  inflating: test/65828/65828.csv    
   creating: test/65829/
  inflating: test/65829/65829.jpg    
  inflating: test/65829/65829.csv    
   creating: test/65830/
  inflating: test/65830/65830.jpg    
  inflating: test/65830/65830.csv    
   creating: test/65831/
  inflating: test/65831/65831.jpg    
  inflating: test/65831/65831.csv    
   creating: test/65832/
  inflating: test/65832/65832.jpg    
  inflating: test/65832/65832.csv    
   creating: test/65833/
  inflating: test/65833/65833.jpg    
  inflating: test

In [5]:
# get sequence feature information

csv_feature_dict = {'내부 습도 1 최고': [25.9, 100.0],
                    '내부 습도 1 최저': [0.0, 100.0],
                    '내부 습도 1 평균': [23.7, 100.0],
                    '내부 온도 1 최고': [3.4, 47.6],
                    '내부 온도 1 최저': [3.3, 47.0],
                    '내부 온도 1 평균': [3.4, 47.3],
                    '내부 이슬점 최고': [0.2, 34.7],
                    '내부 이슬점 최저': [0.0, 34.4],
                    '내부 이슬점 평균': [0.1, 34.5]}

In [6]:
# label encoder, decoder 

train_json = sorted(glob('train/*/*.json'))

labels = []
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample['annotations']['crop']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"
        labels.append(label)

label_encoder = sorted(np.unique(labels))
label_encoder = {key:value for key,value in zip(label_encoder, range(len(label_encoder)))}
label_decoder = {val:key for key, val in label_encoder.items()}

In [7]:
# hyper parameters

opt = dict()

opt['batch_size'] = 16
opt['class_n'] = len(label_encoder)
opt['lr'] = 2e-4
opt['embedding_dim'] = 512
opt['feature_n'] = len(csv_feature_dict)
opt['max_len'] = 300
opt['dropout_rate'] = 0.3
opt['epoch_n'] = 25
opt['vision_pretrain'] = True
opt['worker_n'] = 8
opt['folder'] ='model_weights'
opt['bidirectional'] = True
opt['minmax_dict'] = csv_feature_dict
opt['label_dict'] = label_encoder
opt['enc_name'] = 'resnext50_32x4d'
opt['enc_dim'] = 2048
opt['dec_dim'] = 1024
opt['img_size1'] = 384
opt['img_size2'] = 384
opt['precision'] = 'amp'
opt['seed'] = 42
opt['mix'] = 'cutmix'
opt['mix_prob'] = 0.3
opt['mean'] = [0.485, 0.456, 0.406]
opt['std'] = [0.229, 0.224, 0.225]


device = torch.device("cuda:0")

In [8]:
# fix seed

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(opt['seed'])

In [9]:
# get data

test = sorted(glob('test/*'))

In [10]:
# model

class Encoder(nn.Module):
    def __init__(self, opt):
        super(Encoder, self).__init__()
        self.model = timm.create_model(model_name=opt['enc_name'], 
                                       pretrained=opt['vision_pretrain'], 
                                       num_classes=0)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output


class Decoder(nn.Module):
    def __init__(self, opt):
        super(Decoder, self).__init__()
        self.decoder = nn.GRU(opt['feature_n'], opt['embedding_dim'], bidirectional = opt['bidirectional'])
        self.dense = nn.Linear(2*opt['max_len']*opt['embedding_dim'], opt['dec_dim'])
        
        self.f1 = nn.Linear(opt['enc_dim']+opt['dec_dim'], opt['enc_dim']+opt['dec_dim'])
        self.out = nn.Linear(opt['enc_dim']+opt['dec_dim'], opt['class_n'])
        self.dropout = nn.Dropout(opt['dropout_rate'])
        self.relu = nn.ReLU()

    def init_weight(self):
        torch.nn.init.xavier_uniform_(self.f1.weight)  
        torch.nn.init.xavier_uniform_(self.dense.weight)  
        torch.nn.init.xavier_uniform_(self.out.weight)  


    def forward(self, enc_out, dec_inp):
        dec_out, _ = self.decoder(dec_inp)
        dec_out = self.dense(dec_out.view(dec_out.size(0), -1))

        concat = torch.cat([enc_out, dec_out], dim=1) 
        concat = self.f1(self.relu(concat))
        concat = self.dropout(self.relu(concat))
        output = self.out(concat)
        return output


class CustomModel(nn.Module):
    def __init__(self, opt):
        super(CustomModel, self).__init__()
        self.encoder = Encoder(opt)
        self.decoder = Decoder(opt)
        self.to(device)
        
    def forward(self, img, seq):
        enc_out = self.encoder(img)
        output = self.decoder(enc_out, seq)
        
        return output

In [ ]:
# get model weights

custom_model = CustomModel(opt)

model1_path = glob(opt['folder'] + '/fold1/*.bin')[-1]
model2_path = glob(opt['folder'] + '/fold2/*.bin')[-1]
model3_path = glob(opt['folder'] + '/fold3/*.bin')[-1]
model4_path = glob(opt['folder'] + '/fold4/*.bin')[-1]
model5_path = glob(opt['folder'] + '/fold5/*.bin')[-1]

# fold1 model
model1 = CustomModel(opt)
model1.load_state_dict(torch.load(model1_path, map_location='cpu'))
model1.to(device)
model1.eval()

# fold2 model
model2 = CustomModel(opt)
model2.load_state_dict(torch.load(model2_path, map_location='cpu'))
model2.to(device)
model2.eval()

# fold3 model
model3 = CustomModel(opt)
model3.load_state_dict(torch.load(model3_path, map_location='cpu'))
model3.to(device)
model3.eval()

# fold4 model
model4 = CustomModel(opt)
model4.load_state_dict(torch.load(model4_path, map_location='cpu'))
model4.to(device)
model4.eval()

# fold5 model
model5 = CustomModel(opt)
model5.load_state_dict(torch.load(model5_path, map_location='cpu'))
model5.to(device)
model5.eval()

'model weight is loaded'

In [12]:
# inference dataset

def valTransform():
  return Compose([
                  Resize(opt['img_size1'], opt['img_size2']),
                  Normalize(mean=opt['mean'], std=opt['std'], max_pixel_value=255.0, p=1.0),
                  ToTensorV2(p=1.0),
              ], p=1.)
  
class InferenceDataset(Dataset):
    def __init__(self, opt, files, mode):
        self.files = files
        self.mode = mode
        self.csv_check = [0]*len(self.files)
        self.seq = [None]*len(self.files)
        self.minmax_dict = opt['minmax_dict']
        self.max_len = opt['max_len']
        self.label_encoder = opt['label_dict']

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        file_name = file.split('/')[-1]
        
        if self.csv_check[i] == 0:
            csv_path = f'{file}/{file_name}.csv'
            df = pd.read_csv(csv_path)
            try:
                estiTime1, estiTime2 = df.iloc[0]['측정시각'], df.iloc[1]['측정시각']
            except:
                estiTime1, estiTime2 = 0, 1

            df = df[self.minmax_dict.keys()]
            df = df.replace('-', 0)
            
            if estiTime1==estiTime2 and len(df)>400:
                df = df[0::2].reset_index(drop=True)
                
            
            # minmax-scaling
            for col in df.columns:
                df[col] = df[col].astype(float) - self.minmax_dict[col][0]
                df[col] = df[col] / (self.minmax_dict[col][1]-self.minmax_dict[col][0])

            # zero-padding
            pad = np.zeros((self.max_len, len(df.columns)))
            length = min(self.max_len, len(df))
            pad[-length:] = df.to_numpy()[-length:]

            # transpose-to-sequential-data
            seq = torch.tensor(pad, dtype=torch.float32)
            self.seq[i] = seq
            self.csv_check[i] = 1
        else:
            seq = self.seq[i]
        
        image_path = f'{file}/{file_name}.jpg'
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.uint8)
        img = valTransform()(image=img)['image'] 

        if self.mode == 'val':
            json_path = f'{file}/{file_name}.json'
            with open(json_path, 'r') as f:
                json_file = json.load(f)
            
            crop = json_file['annotations']['crop']
            disease = json_file['annotations']['disease']
            risk = json_file['annotations']['risk']
            label = torch.tensor(self.label_encoder[f'{crop}_{disease}_{risk}'], dtype=torch.long)
            
            return img, seq, label
        else:
            return img, seq

In [13]:
# predict function

def predict(models, loader, mode):
    model1, model2, model3, model4, model5 = models

    preds = []
    for bi, data in enumerate(tqdm(loader)):
        data = [x.to(device) for x in data]
        if mode=='val':
            img, seq, label = data
        else:
            img, seq = data
        output1 = nn.Softmax(dim=-1)(model1(img, seq))
        output2 = nn.Softmax(dim=-1)(model2(img, seq))
        output3 = nn.Softmax(dim=-1)(model3(img, seq))
        output4 = nn.Softmax(dim=-1)(model4(img, seq))
        output5 = nn.Softmax(dim=-1)(model5(img, seq))

        output = output1 + output2 + output3 + output4 + output5
        pred = torch.argmax(output, dim=1).cpu().tolist()
        preds.extend(pred)
    return preds

In [ ]:
# get test preds

models = [model1, model2, model3, model4, model5]

test_dataset = InferenceDataset(opt, test, mode='test')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=2*opt['batch_size'], num_workers=opt['worker_n'], shuffle=False)

with torch.no_grad():
    preds = predict(models, test_dataloader, mode='test')

In [ ]:
preds = np.array([label_decoder[int(x)] for x in preds])

In [20]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/농업 환경 변화에 따른 작물 병해 진단 AI 경진대회/data/sample_submission.csv')
submission['label'] = preds

In [19]:
submission.to_csv('cnn2rnn(resnext-5fold-ensemble)_submission.csv', index=False)